In [ ]:
import time

from audio_processing import AudioRecorder, SileroVAD
from whisper_transcriber import WhisperTranscriber

In [ ]:
# 初始化錄音器與 Whisper 轉錄
recorder = AudioRecorder()
transcriber = WhisperTranscriber(model_size="medium", device="cuda", compute_type="int8")
vad = SileroVAD()

# 列出裝置並開始錄音
#recorder.list_audio_devices()
recorder.get_default_device()

transcribe_text = ""

In [ ]:
recorder.start_recording()

try:
    while True:
        recorder.record_audio()
        
        if recorder.audio_queue.qsize() * recorder.chunk_size >= recorder.rate * recorder.threshold_seconds:
            audio_accumulated_np = recorder.process_audio_queue()
            audio_accumulated_np = audio_accumulated_np / 32768.0
            print("Out of threshold, process audio queue")
            if vad.is_speech(audio_accumulated_np):
                stt_result = transcriber.process_audio_chunk(audio_accumulated_np)
                if stt_result:
                    print(stt_result)
                    transcribe_text += " ".join(stt_result)
            else:
                print("Silence detected")
                time.sleep(0.1)
                # process the transcribe_text
                if transcribe_text:
                    print("Transcribe text:", transcribe_text)
                    transcribe_text = ""
                

except KeyboardInterrupt:
    recorder.stop_recording()